In [ ]:
import pyJMT as jmt
import math
import numpy as np

def mm1_task_classification_system():
    # Create a new network model
    model = jmt.Network("M/M/1 Task Classification System")
    
    # Define components
    source = jmt.Source(model, "Task Arrivals")
    classifier = jmt.Router(model, "Task Classifier")
    small_queue = jmt.Queue(model, "Small Task Queue", jmt.SchedStrategy.FCFS)
    medium_queue = jmt.Queue(model, "Medium Task Queue", jmt.SchedStrategy.FCFS)
    large_queue = jmt.Queue(model, "Large Task Queue", jmt.SchedStrategy.FCFS)
    
    # Updated number of PMs for each VM category
    N = 15 # Increased from 10
    small_pms = []
    medium_pms = []
    large_pms = []
    
    # Create PMs for each category
    for i in range(N):
        small_pms.append(jmt.Queue(model, f"Small_PM_{i}", jmt.SchedStrategy.FCFS))
        medium_pms.append(jmt.Queue(model, f"Medium_PM_{i}", jmt.SchedStrategy.FCFS))
        large_pms.append(jmt.Queue(model, f"Large_PM_{i}", jmt.SchedStrategy.FCFS))
    
    sink = jmt.Sink(model, "Task Completed")
    
    # Define arrival thresholds for classification
    t1 = 2.5  # Slightly reduced threshold for better distribution
    t2 = 5.5  

    # Define task classes
    small_task = jmt.OpenClass(model, "Small Task")
    medium_task = jmt.OpenClass(model, "Medium Task")
    large_task = jmt.OpenClass(model, "Large Task")
    
    # Adjusted arrival rates (λ)
    lambda_small = 7.0  # Increased from 5.0
    lambda_medium = 4.5  # Increased from 3.0
    lambda_large = 2.5  # Increased from 1.0
    
    # Adjusted service rates (μ)
    mu = 30000  # Increased from 25000
    mu_small = 50
    mu_medium = 100
    mu_large = 180
    
    # Updated PM parameters
    K = 120  # Increased max tasks per PM from 100
    m = 20   # Increased VMs per PM from 15
    
    # Define Poisson arrivals 
    source.setArrival(small_task, jmt.Exp(lambda_small))
    source.setArrival(medium_task, jmt.Exp(lambda_medium))
    source.setArrival(large_task, jmt.Exp(lambda_large))
    
    # Define exponential service times for the main queues
    small_queue.setService(small_task,jmt.Exp(mu_small))
    medium_queue.setService(medium_task,jmt.Exp(mu_medium))
    large_queue.setService(large_task, jmt.Exp(mu_large))
    
    # Set number of servers for each queue (M/M/1 means 1 server)
    small_queue.setNumberOfServers(1)
    medium_queue.setNumberOfServers(1)
    large_queue.setNumberOfServers(1)
    
    # Configure service times for each PM based on VM type
    for pm in small_pms:
        pm.setService(small_task,jmt.Exp(mu_small))
        pm.setNumberOfServers(m)  # Set increased number of VMs per PM
        pm.setCapacity(K)  # Set max tasks per PM
    
    for pm in medium_pms:
        pm.setService(medium_task,jmt.Exp(mu_medium))
        pm.setNumberOfServers(m)
        pm.setCapacity(K)
    
    for pm in large_pms:
        pm.setService(large_task,jmt.Exp(mu_large))#Changed from jmt.Exp(mu_large) to mu_large
        pm.setNumberOfServers(m)
        pm.setCapacity(K)
    
    # Configure task classifier routing
    classifier.setRouting(small_task, jmt.RoutingStrategy.PROB)
    classifier.setRouting(medium_task, jmt.RoutingStrategy.PROB)
    classifier.setRouting(large_task, jmt.RoutingStrategy.PROB)
    
    # Route each task to its corresponding queue with 100% probability
    classifier.setProbRouting(small_task, small_queue, 1.0)
    classifier.setProbRouting(medium_task, medium_queue, 1.0)
    classifier.setProbRouting(large_task, large_queue, 1.0)
    
    # Configure round-robin routing from queues to PMs
    small_queue.setRouting(small_task, jmt.RoutingStrategy.RROBIN)
    medium_queue.setRouting(medium_task, jmt.RoutingStrategy.RROBIN)
    large_queue.setRouting(large_task, jmt.RoutingStrategy.RROBIN)
    
    # Connect small queue to small PMs
    for pm in small_pms:
        model.addLink(small_queue, pm)
        model.addLink(pm, sink)
    
    # Connect medium queue to medium PMs
    for pm in medium_pms:
        model.addLink(medium_queue, pm)
        model.addLink(pm, sink)
    
    # Connect large queue to large PMs
    for pm in large_pms:
        model.addLink(large_queue, pm)
        model.addLink(pm, sink)
    
    # Connect source to classifier and classifier to queues
    model.addLinks([
        (source, classifier),
        (classifier, small_queue),
        (classifier, medium_queue),
        (classifier, large_queue)
    ])
    
    # Save and run the model
    model.saveNamedJsimg("task_classification_system")

    model.jsimgOpen()
    model.saveResultsFileNamed("task_system_results")
    results = model.getResults()
    
    return model, results
if __name__ == "__main__":
    model, results = mm1_task_classification_system()